In [0]:
#%fs ls /FileStore/dataset

In [0]:
#import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
#create spark session
spark = SparkSession.builder.appName('Domestic Violence').getOrCreate()

In [0]:
#Mapping the path of the file
FILEPATH = '/FileStore/dataset/Reporte_Delito_Violencia_Intrafamiliar_Polic_a_Nacional.csv'

In [0]:
#Schema for Dataframe
SCHEMA = StructType([
    StructField('DEPARTAMENTO', StringType(), True),
    StructField('MUNICIPIO', StringType(), True),
    StructField('CODIGO', StringType(), True),
    StructField('ARMAS MEDIOS', StringType(), True),
    StructField('FECHA HECHO', StringType(), True),
    StructField('GENERO', StringType(), True),
    StructField('GRUPO ETARIO', StringType(), True),
    StructField('CANTIDAD', IntegerType(), True),
])

In [0]:
# Load file
raw_df = spark.read.csv(path=FILEPATH, header=True, schema=SCHEMA, sep=',')
raw_df.show()

+------------+-----------------+--------+--------------------+-----------+---------+------------+--------+
|DEPARTAMENTO|        MUNICIPIO|  CODIGO|        ARMAS MEDIOS|FECHA HECHO|   GENERO|GRUPO ETARIO|CANTIDAD|
+------------+-----------------+--------+--------------------+-----------+---------+------------+--------+
|   ATLÁNTICO|BARRANQUILLA (CT)| 8001000|ARMA BLANCA / COR...|  1/01/2010|MASCULINO|     ADULTOS|       1|
|      BOYACÁ|          DUITAMA|15238000|ARMA BLANCA / COR...|  1/01/2010| FEMENINO|     ADULTOS|       1|
|     CAQUETÁ|      PUERTO RICO|18592000|ARMA BLANCA / COR...|  1/01/2010|MASCULINO|     ADULTOS|       1|
|    CASANARE|             MANÍ|85139000|ARMA BLANCA / COR...|  1/01/2010| FEMENINO|     ADULTOS|       1|
|CUNDINAMARCA| BOGOTÁ D.C. (CT)|11001000|ARMA BLANCA / COR...|  1/01/2010| FEMENINO|     ADULTOS|       1|
|       SUCRE|   SINCELEJO (CT)|70001000|ARMA BLANCA / COR...|  1/01/2010| FEMENINO|     ADULTOS|       1|
|       VALLE|        CALI (CT)|76001

In [0]:
raw_df.printSchema()

root
 |-- DEPARTAMENTO: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- CODIGO: string (nullable = true)
 |-- ARMAS MEDIOS: string (nullable = true)
 |-- FECHA HECHO: string (nullable = true)
 |-- GENERO: string (nullable = true)
 |-- GRUPO ETARIO: string (nullable = true)
 |-- CANTIDAD: integer (nullable = true)



In [0]:
#summary Dataframe
describe_df = raw_df.describe()
describe_df.display()

summary,DEPARTAMENTO,MUNICIPIO,CODIGO,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD
count,476970,476970,476970,476968,476970,476967,475355,476970
mean,null,null,3.775064627790055E7,null,44277.5,null,null,1.7077635910015305
stddev,null,null,2.7493718706527133E7,null,47.216825088399965,null,null,3.338647115163399
min,AMAZONAS,ABEJORRAL,11001000,-,1/01/2010,-,ADOLESCENTES,1
max,VICHADA,ÚTICA,NO REPORTA,SIN EMPLEO DE ARMAS,9/12/2020,NO REPORTA,NO REPORTA,130


In [0]:
#Rename columns
name_columns = ['department', 'municipality', 'code', 'armaments', 'date', 'gender', 'group', 'quantity']
rename_columns_df = raw_df.toDF(*name_columns)

rename_columns_df.show()

+------------+-----------------+--------+--------------------+---------+---------+-------+--------+
|  department|     municipality|    code|           armaments|     date|   gender|  group|quantity|
+------------+-----------------+--------+--------------------+---------+---------+-------+--------+
|   ATLÁNTICO|BARRANQUILLA (CT)| 8001000|ARMA BLANCA / COR...|1/01/2010|MASCULINO|ADULTOS|       1|
|      BOYACÁ|          DUITAMA|15238000|ARMA BLANCA / COR...|1/01/2010| FEMENINO|ADULTOS|       1|
|     CAQUETÁ|      PUERTO RICO|18592000|ARMA BLANCA / COR...|1/01/2010|MASCULINO|ADULTOS|       1|
|    CASANARE|             MANÍ|85139000|ARMA BLANCA / COR...|1/01/2010| FEMENINO|ADULTOS|       1|
|CUNDINAMARCA| BOGOTÁ D.C. (CT)|11001000|ARMA BLANCA / COR...|1/01/2010| FEMENINO|ADULTOS|       1|
|       SUCRE|   SINCELEJO (CT)|70001000|ARMA BLANCA / COR...|1/01/2010| FEMENINO|ADULTOS|       1|
|       VALLE|        CALI (CT)|76001000|ARMA BLANCA / COR...|1/01/2010| FEMENINO|ADULTOS|       2|


In [0]:
#Checking for null columns
department_nulls = rename_columns_df.filter(col('department').isNull()).count()
municipality_nulls = rename_columns_df.filter(col('municipality').isNull()).count()
code_nulls = rename_columns_df.filter(col('code').isNull()).count()
armaments_nulls = rename_columns_df.filter(col('armaments').isNull()).count()
date_nulls = rename_columns_df.filter(col('date').isNull()).count()
gender_nulls = rename_columns_df.filter(col('gender').isNull()).count()
group_nulls = rename_columns_df.filter(col('group').isNull()).count()
quantity_nulls = rename_columns_df.filter(col('quantity').isNull()).count()

print('department: ', department_nulls)
print('municipality: ', municipality_nulls)
print('code: ', code_nulls)
print('armaments: ', armaments_nulls)
print('date: ', date_nulls)
print('gender: ', gender_nulls)
print('group: ', group_nulls)
print('quantity: ', quantity_nulls)

department:  0
municipality:  0
code:  0
armaments:  2
date:  0
gender:  3
group:  1615
quantity:  0


In [0]:
#Replace nulls - column armaments
replace_nulls_armaments_df = rename_columns_df.fillna('SIN EMPLEO DE ARMAS', subset=['armaments'])

#-- checkings
armaments_nulls = replace_nulls_armaments_df.filter(col('armaments').isNull()).count()
print('armaments: ', armaments_nulls)

armaments:  0


In [0]:
#Replace nulls - column gender
# -- calculate mode
mode_gender = replace_nulls_armaments_df.groupBy('gender').count().orderBy('count', ascending=False).first()['gender']
print(mode_gender)

# -- replace
replace_nulls_gender_df = replace_nulls_armaments_df.fillna(mode_gender, subset=['gender'])

#-- checkings
gender_nulls = replace_nulls_gender_df.filter(col('gender').isNull()).count()
print('gender: ', gender_nulls)


FEMENINO
gender:  0


In [0]:
#Replace nulls - column group
replace_nulls_df = replace_nulls_gender_df.fillna('NO REPORTA', subset=['group'])

#-- checkings
group_nulls = replace_nulls_df.filter(col('group').isNull()).count()
print('group: ', group_nulls)

group:  0


In [0]:
#Checking for null columns
department_nulls = replace_nulls_df.filter(col('department').isNull()).count()
municipality_nulls = replace_nulls_df.filter(col('municipality').isNull()).count()
code_nulls = replace_nulls_df.filter(col('code').isNull()).count()
armaments_nulls = replace_nulls_df.filter(col('armaments').isNull()).count()
date_nulls = replace_nulls_df.filter(col('date').isNull()).count()
gender_nulls = replace_nulls_df.filter(col('gender').isNull()).count()
group_nulls = replace_nulls_df.filter(col('group').isNull()).count()
quantity_nulls = replace_nulls_df.filter(col('quantity').isNull()).count()

print('department: ', department_nulls)
print('municipality: ', municipality_nulls)
print('code: ', code_nulls)
print('armaments: ', armaments_nulls)
print('date: ', date_nulls)
print('gender: ', gender_nulls)
print('group: ', group_nulls)
print('quantity: ', quantity_nulls)

department:  0
municipality:  0
code:  0
armaments:  0
date:  0
gender:  0
group:  0
quantity:  0


In [0]:
#Change format column date
cast_datecolumns_df = replace_nulls_df.withColumn('date', expr("date_format(to_date(date, 'd/M/yyyy'), 'yyyy-MM-dd')"))
cast_datecolumns_df.show(5)

+------------+-----------------+--------+--------------------+----------+---------+-------+--------+
|  department|     municipality|    code|           armaments|      date|   gender|  group|quantity|
+------------+-----------------+--------+--------------------+----------+---------+-------+--------+
|   ATLÁNTICO|BARRANQUILLA (CT)| 8001000|ARMA BLANCA / COR...|2010-01-01|MASCULINO|ADULTOS|       1|
|      BOYACÁ|          DUITAMA|15238000|ARMA BLANCA / COR...|2010-01-01| FEMENINO|ADULTOS|       1|
|     CAQUETÁ|      PUERTO RICO|18592000|ARMA BLANCA / COR...|2010-01-01|MASCULINO|ADULTOS|       1|
|    CASANARE|             MANÍ|85139000|ARMA BLANCA / COR...|2010-01-01| FEMENINO|ADULTOS|       1|
|CUNDINAMARCA| BOGOTÁ D.C. (CT)|11001000|ARMA BLANCA / COR...|2010-01-01| FEMENINO|ADULTOS|       1|
+------------+-----------------+--------+--------------------+----------+---------+-------+--------+
only showing top 5 rows



In [0]:
# Add columns year, month, day

add_columns_df = cast_datecolumns_df.withColumn('year', year(col('date')))\
    .withColumn('month', date_format(col('date'), 'MMMM'))\
    .withColumn('day', date_format(col('date'), 'EEEE'))
add_columns_df.show(5)

+------------+-----------------+--------+--------------------+----------+---------+-------+--------+----+-------+------+
|  department|     municipality|    code|           armaments|      date|   gender|  group|quantity|year|  month|   day|
+------------+-----------------+--------+--------------------+----------+---------+-------+--------+----+-------+------+
|   ATLÁNTICO|BARRANQUILLA (CT)| 8001000|ARMA BLANCA / COR...|2010-01-01|MASCULINO|ADULTOS|       1|2010|January|Friday|
|      BOYACÁ|          DUITAMA|15238000|ARMA BLANCA / COR...|2010-01-01| FEMENINO|ADULTOS|       1|2010|January|Friday|
|     CAQUETÁ|      PUERTO RICO|18592000|ARMA BLANCA / COR...|2010-01-01|MASCULINO|ADULTOS|       1|2010|January|Friday|
|    CASANARE|             MANÍ|85139000|ARMA BLANCA / COR...|2010-01-01| FEMENINO|ADULTOS|       1|2010|January|Friday|
|CUNDINAMARCA| BOGOTÁ D.C. (CT)|11001000|ARMA BLANCA / COR...|2010-01-01| FEMENINO|ADULTOS|       1|2010|January|Friday|
+------------+-----------------+

In [0]:
# Remove column date
df_filter = add_columns_df.drop(col('date'))
df_filter.show(5)

+------------+-----------------+--------+--------------------+---------+-------+--------+----+-------+------+
|  department|     municipality|    code|           armaments|   gender|  group|quantity|year|  month|   day|
+------------+-----------------+--------+--------------------+---------+-------+--------+----+-------+------+
|   ATLÁNTICO|BARRANQUILLA (CT)| 8001000|ARMA BLANCA / COR...|MASCULINO|ADULTOS|       1|2010|January|Friday|
|      BOYACÁ|          DUITAMA|15238000|ARMA BLANCA / COR...| FEMENINO|ADULTOS|       1|2010|January|Friday|
|     CAQUETÁ|      PUERTO RICO|18592000|ARMA BLANCA / COR...|MASCULINO|ADULTOS|       1|2010|January|Friday|
|    CASANARE|             MANÍ|85139000|ARMA BLANCA / COR...| FEMENINO|ADULTOS|       1|2010|January|Friday|
|CUNDINAMARCA| BOGOTÁ D.C. (CT)|11001000|ARMA BLANCA / COR...| FEMENINO|ADULTOS|       1|2010|January|Friday|
+------------+-----------------+--------+--------------------+---------+-------+--------+----+-------+------+
only showi

In [0]:
#Filter sex column
filter_gender_df = df_filter.withColumn('gender', when( lower(col('gender')) == 'masculino', 'M' )\
                    .when( lower(col('gender')) == 'femenino', 'F' ).otherwise(col('gender')) )
filter_gender_df.show(5)

+------------+-----------------+--------+--------------------+------+-------+--------+----+-------+------+
|  department|     municipality|    code|           armaments|gender|  group|quantity|year|  month|   day|
+------------+-----------------+--------+--------------------+------+-------+--------+----+-------+------+
|   ATLÁNTICO|BARRANQUILLA (CT)| 8001000|ARMA BLANCA / COR...|     M|ADULTOS|       1|2010|January|Friday|
|      BOYACÁ|          DUITAMA|15238000|ARMA BLANCA / COR...|     F|ADULTOS|       1|2010|January|Friday|
|     CAQUETÁ|      PUERTO RICO|18592000|ARMA BLANCA / COR...|     M|ADULTOS|       1|2010|January|Friday|
|    CASANARE|             MANÍ|85139000|ARMA BLANCA / COR...|     F|ADULTOS|       1|2010|January|Friday|
|CUNDINAMARCA| BOGOTÁ D.C. (CT)|11001000|ARMA BLANCA / COR...|     F|ADULTOS|       1|2010|January|Friday|
+------------+-----------------+--------+--------------------+------+-------+--------+----+-------+------+
only showing top 5 rows



In [0]:
df = filter_gender_df.withColumnRenamed('group', 'age_category')
df.show()

+------------+-----------------+--------+--------------------+------+------------+--------+----+-------+------+
|  department|     municipality|    code|           armaments|gender|age_category|quantity|year|  month|   day|
+------------+-----------------+--------+--------------------+------+------------+--------+----+-------+------+
|   ATLÁNTICO|BARRANQUILLA (CT)| 8001000|ARMA BLANCA / COR...|     M|     ADULTOS|       1|2010|January|Friday|
|      BOYACÁ|          DUITAMA|15238000|ARMA BLANCA / COR...|     F|     ADULTOS|       1|2010|January|Friday|
|     CAQUETÁ|      PUERTO RICO|18592000|ARMA BLANCA / COR...|     M|     ADULTOS|       1|2010|January|Friday|
|    CASANARE|             MANÍ|85139000|ARMA BLANCA / COR...|     F|     ADULTOS|       1|2010|January|Friday|
|CUNDINAMARCA| BOGOTÁ D.C. (CT)|11001000|ARMA BLANCA / COR...|     F|     ADULTOS|       1|2010|January|Friday|
|       SUCRE|   SINCELEJO (CT)|70001000|ARMA BLANCA / COR...|     F|     ADULTOS|       1|2010|January|

In [0]:
# Sort columns
columns_order = ['code', 'year', 'month', 'day', 'department', 'municipality', 'armaments', 'gender', 'age_category', 'quantity']
df = df.select(*columns_order)
df.show()

+--------+----+-------+------+------------+-----------------+--------------------+------+------------+--------+
|    code|year|  month|   day|  department|     municipality|           armaments|gender|age_category|quantity|
+--------+----+-------+------+------------+-----------------+--------------------+------+------------+--------+
| 8001000|2010|January|Friday|   ATLÁNTICO|BARRANQUILLA (CT)|ARMA BLANCA / COR...|     M|     ADULTOS|       1|
|15238000|2010|January|Friday|      BOYACÁ|          DUITAMA|ARMA BLANCA / COR...|     F|     ADULTOS|       1|
|18592000|2010|January|Friday|     CAQUETÁ|      PUERTO RICO|ARMA BLANCA / COR...|     M|     ADULTOS|       1|
|85139000|2010|January|Friday|    CASANARE|             MANÍ|ARMA BLANCA / COR...|     F|     ADULTOS|       1|
|11001000|2010|January|Friday|CUNDINAMARCA| BOGOTÁ D.C. (CT)|ARMA BLANCA / COR...|     F|     ADULTOS|       1|
|70001000|2010|January|Friday|       SUCRE|   SINCELEJO (CT)|ARMA BLANCA / COR...|     F|     ADULTOS|  

In [0]:
# Number of cases by year
cases_by_date = df.groupBy('year').count().withColumnRenamed('count', 'total_cases')\
                .filter(col('year').isNotNull()).orderBy('total_cases', ascending=False)
cases_by_date.display()

year,total_cases
2020,102616
2019,52924
2017,50620
2016,50055
2018,48098
2015,45181
2014,30045
2021,22416
2013,21982
2012,20602


In [0]:
# Number of cases by Department 
cases_department = df.groupBy('department').count().withColumnRenamed('count', 'total_cases').orderBy('total_cases', ascending=False).limit(10)
cases_department.display()

department,total_cases
CUNDINAMARCA,88100
ANTIOQUIA,57288
VALLE,44909
SANTANDER,38493
BOYACÁ,26920
ATLÁNTICO,17516
TOLIMA,17007
HUILA,16547
META,16223
BOLÍVAR,15954


In [0]:
# Number of cases by gender 
cases_gender = df.where(col('gender') != '-').groupBy('gender').count().withColumnRenamed('count', 'total_cases')\
                .orderBy('total_cases', ascending=False)
cases_gender.display()

gender,total_cases
F,366910
M,109759
NO REPORTA,294


In [0]:
#Armaments most used in 2021
cases_armaments = df.where(col('armaments') != '-').where(col('year') == '2021').groupBy('armaments').count()\
                    .withColumnRenamed('count', 'total_cases').orderBy('total_cases', ascending=False)
cases_armaments.display()

armaments,total_cases
SIN EMPLEO DE ARMAS,12091
CONTUNDENTES,9323
ARMA BLANCA / CORTOPUNZANTE,902
ARMA DE FUEGO,98
NO REPORTADO,2
